In [ ]:
import geopandas as gpd
soa_shape_map_path = '/home/mahdi/data/PRISM_tmin_stable_4kmD2_20160612_nc/doc_2024-03-26_15-21-36.gridpoint_2000.shp/.gridpoint_2000.shp.shp'
sub_basin_path = '/home/mahdi/data/PRISM_tmin_stable_4kmD2_20160612_nc/central_valley_3857/central_valley_3857.shp'

soa_shape_map = gpd.read_file(soa_shape_map_path)
sub_basin_map = gpd.read_file(sub_basin_path)

soa_shape_map_geo = soa_shape_map.to_crs(epsg=4326)
sub_basin_map_geo = sub_basin_map.to_crs(epsg=4326)

sub_basin_1 = sub_basin_map_geo[sub_basin_map_geo['Subregion_'] == 1]




In [ ]:
clipped_points = gpd.clip(soa_shape_map_geo, sub_basin_1)

lat_lon_tuples = list(zip(clipped_points.geometry.y, clipped_points.geometry.x))


In [ ]:
import datetime

def generate_time_series(start_date, end_date, freq):
    """
    Generate a time series of dates in the format "YYYYMMDD".

    Args:
        start_date (str): The start date in the format "YYYY-MM-DD".
        end_date (str): The end date in the format "YYYY-MM-DD".
        freq (str): The frequency of the time series. Possible values are
            "D" (daily), "W" (weekly), "M" (monthly), etc.

    Returns:
        list: A list of dates in the format "YYYYMMDD".
    """
    date_series = []
    start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    current_date = start

    while current_date <= end:
        date_series.append(current_date.strftime("%Y%m%d"))
        current_date += datetime.timedelta(days=1) if freq == "D" else datetime.timedelta(weeks=1) if freq == "W" else datetime.timedelta(days=30.4375)

    return date_series

In [ ]:
import requests
import pandas as pd

def download_file(url):
  """Downloads a file from the given URL and saves it locally."""
  local_filename = url.split('/')[-1]
  # NOTE the stream=True parameter below
  with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(local_filename, 'wb') as f:
      for chunk in r.iter_content(chunk_size=8192): 
        f.write(chunk)
  return local_filename


def download_prism_data(lat_lon_tuples, output_folder):
  """Downloads PIRISM data for a list of latitude and longitude tuples.

  Args:
    lat_lon_tuples: A list of tuples containing (latitude, longitude) coordinates.
    output_folder: The folder where downloaded data files will be saved.
  """
  session = requests.Session()
  base_url = "https://prism.oregonstate.edu/explorer/dataexplorer/rpc.php"
  data = {
    "spares": "4km",
    "interp": "0",
    "stats": "ppt tmin tmean tmax",  # You can customize the variables here
    "units": "si",
    "range": "daily",
    "start": "19910101",
    "end": "20201231",
    "stability": "stable",
    "call": "pp/daily_timeseries",
    "proc": "gridserv"
  }
  headers = {
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Accept-Encoding": "gzip, deflate, br",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0",
    "DNT": "1",
    "Sec-GPC": "1",
  }

  for lat, lon in lat_lon_tuples:
    data["lat"] = str(lat)
    data["lon"] = str(lon)
    response = session.post(base_url, data=data, headers=headers)
    res_json = response.json()

    
    download_url = "https://prism.oregonstate.edu/explorer/tmp/" + res_json["result"]["csv"]
    filename = f"{output_folder}/{lat}_{lon}.csv"  # Customize filename format
    download_file(download_url)
    print(f"Downloaded data for ({lat}, {lon}) to {filename}")
    


In [ ]:
# Example usage
output_folder = "/home/mahdi/data/export"  # Change this if needed
download_prism_data(lat_lon_tuples, output_folder)

In [ ]:
import requests
import pandas as pd

def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filename






for lat, lon in lat_lon_tuples:
    session = requests.Session()
    response = session.post("https://prism.oregonstate.edu/explorer/dataexplorer/rpc.php",
                            data={
                                "spares": "4km",
                                "interp": "0",
                                "stats": "ppt tmin tmean tmax",
                                "units": "si",
                                "range": "daily",
                                "start": "19910101",
                                "end": "20201231",
                                "stability": "stable",
                                "lon": str(lon),
                                "lat": str(lat),
                                "elev": "8",
                                "call": "pp/daily_timeseries",
                                "proc": "gridserv"
                            },
                            headers={
                                "Accept": "application/json, text/javascript, */*; q=0.01",
                                "Accept-Encoding": "gzip, deflate, br",
                                "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
                                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0",
                                "DNT": "1",
                                "Sec-GPC": "1",
                            },
                            )

    res_json = response.json()
    print(res_json)
    
    download_url = "https://prism.oregonstate.edu/explorer/tmp/" + res_json["result"]["csv"]

    download_file(download_url)
        

In [2]:
import os
import sys
sys.path.append('/home/mahdi/data/pyPRISMClimate')
from pyPRISMClimate.quick_tools import *

In [4]:

data = get_prism_dailys(
    variable ="tmean",
    min_date="1991-01-01",
    max_date="2020-12-31",
    dest_path="/home/mahdi/data/export"
)

In [16]:

 
from osgeo import gdal

# Open the BIL file
dataset = gdal.Open("/home/mahdi/data/export/PRISM_tmean_stable_4kmD2_20201227_bil.bil")

In [19]:
# Get the number of bands and data type
num_bands = dataset.RasterCount
data_type = dataset.GetRasterBand(1).DataType
# Read the data as a NumPy array
data = dataset.GetRasterBand(1).ReadAsArray()
# (Optional) Get geotransform information
geotransform = dataset.GetGeoTransform()
geotransform

(-125.0208333333335,
 0.041666666667,
 0.0,
 49.9374999999995,
 0.0,
 -0.041666666667)